In [2]:
import sys
sys.path.append("../../../")

In [3]:
from onsager import crystal, supercell, cluster
import numpy as np
import scipy as sp
import itertools
import Transitions
import Cluster_Expansion
import MC_JIT
import time
from tqdm import tqdm
from numba import jit, int64, float64

## Set up the vector cluster expander

In [19]:
import pickle

# See if a cluster expander already exists
try:
    with open("ClusterExpander.pkl", "rb") as fl:
        print("Pre-created Cluster expander found")
        VclusExp = pickle.load(fl)

# Otherwise make one
except:
    N_units = 8 # No. of unit cells along each axis in the supercell
    NSpec = 5
    MaxOrder = 3
    MaxOrderTrans = 3

    a0 = 1.0
    cut = 1.01*a0*np.sqrt(3)/2  # Nearest neighbor cutoff

    crys = crystal.Crystal.BCC(a0, chemistry="A")
    jnetBCC = crys.jumpnetwork(0, cut)

    superlatt = N_units * np.eye(3, dtype=int)
    superBCC = supercell.ClusterSupercell(crys, superlatt)
    # Since the "spectator" tuple in the argument is left empty, all
    # sites of chemistry 0 will be mobile sites.

    vacsite = cluster.ClusterSite((0, 0), np.zeros(3, dtype=int))
    vacsiteInd = superBCC.index(np.zeros(3, dtype=int), (0, 0))[0]
    assert vacsiteInd == 0

    clusexp = cluster.makeclusters(crys, 1.01*a0, MaxOrder)
    Tclusexp = cluster.makeclusters(crys, 1.01*a0, MaxOrderTrans)

    VclusExp = Cluster_Expansion.VectorClusterExpansion(superBCC, clusexp, Tclusexp, jnetBCC,
                                                             NSpec, vacsite, MaxOrder,
                                                             MaxOrderTrans, NoTrans=False)

    print("Done setting up")
    
    # save it for later
    with open("ClusterExpander.pkl", "wb") as fl:
        pickle.dump(VclusExp, fl)

Generated clusters with species: 0.4390
Generated vector basis data 0.9689
Generated Indexing data 0.1643
Generated Interaction data 74.3339
Done setting up


In [20]:
len(VclusExp.vecClus)

132

In [6]:
# First, we have to generate all the arrays
# Lattice gas - set all energies to zero
Energies = np.zeros(len(VclusExp.SpecClusters))
KRAEnergies = [np.zeros(len(val)) for (key, val) in VclusExp.KRAexpander.clusterSpeciesJumps.items()]

numSitesInteracts, SupSitesInteracts, SpecOnInteractSites, Interaction2En, numVecsInteracts,\
VecsInteracts, VecGroupInteracts, numInteractsSiteSpec, SiteSpecInterArray, vacSiteInd,\
InteractionIndexDict, InteractionRepClusDict, Index2InteractionDict, repClustCounter,\
Interact2RepClusArray, Interact2SymClassArray\
= VclusExp.makeJitInteractionsData(Energies)

TsInteractIndexDict, Index2TSinteractDict, numSitesTSInteracts, TSInteractSites, TSInteractSpecs, \
jumpFinSites, jumpFinSpec, FinSiteFinSpecJumpInd, numJumpPointGroups, numTSInteractsInPtGroups, \
JumpInteracts, Jump2KRAEng =\
    VclusExp.KRAexpander.makeTransJitData(KRAEnergies)

Done Indexing interactions : 0.8431355953216553
Done with chemical and symmetry class data for interactions : 0.6313838958740234
Done with vector and energy data for interactions : 1.081432819366455


In [7]:
# Make the MC class
N_units=8
MCJit = MC_JIT.MCSamplerClass(
    numSitesInteracts, SupSitesInteracts, SpecOnInteractSites, Interaction2En, Interact2RepClusArray,
    Interact2SymClassArray, numVecsInteracts, VecsInteracts, VecGroupInteracts, numInteractsSiteSpec,
    SiteSpecInterArray, numSitesTSInteracts, TSInteractSites, TSInteractSpecs, jumpFinSites, jumpFinSpec,
    FinSiteFinSpecJumpInd, numJumpPointGroups, numTSInteractsInPtGroups, JumpInteracts, Jump2KRAEng
)
# Make the KMC class to generate trajectories
siteIndtoR, RtoSiteInd = VclusExp.makeSiteIndToSite()
KMCJit = MC_JIT.KMC_JIT(numSitesInteracts, SupSitesInteracts, SpecOnInteractSites, Interaction2En,
                        Interact2RepClusArray, Interact2SymClassArray, numVecsInteracts, VecsInteracts,
                        VecGroupInteracts, numInteractsSiteSpec, SiteSpecInterArray, numSitesTSInteracts,
                        TSInteractSites, TSInteractSpecs, jumpFinSites, jumpFinSpec, FinSiteFinSpecJumpInd,
                        numJumpPointGroups, numTSInteractsInPtGroups, JumpInteracts, Jump2KRAEng,
                        siteIndtoR, RtoSiteInd, N_units)

In [8]:
# Set up a random binary alloy
NSpec = 3
state = np.zeros(VclusExp.Nsites, dtype=int)
c0 = 0.75
N0 = int(c0*VclusExp.Nsites)
for i in range(N0):
    state[i] = 0
for i in range(N0, VclusExp.Nsites):
    state[i] = 1

# permute it
state = np.random.permutation(state)
# put the vacancy at the origin
state[vacSiteInd] = NSpec - 1
specs, counts = np.unique(state, return_counts=True)
print("N0:{}, N1:{}, Nvac:{}".format(counts[0], counts[1], counts[2]))

N0:383, N1:128, Nvac:1


In [9]:
jList, dxList = VclusExp.KRAexpander.ijList, VclusExp.KRAexpander.dxList

In [10]:
# expand
specRates = np.array([0.001, 1.0])
NVclus = len(VclusExp.vecVec)
offsc = MC_JIT.GetOffSite(state, numSitesInteracts, SupSitesInteracts, SpecOnInteractSites)

In [11]:
# Let's make the NoTrans vector cluster arrays
len(VclusExp.Interact2VecInteract)

144384

In [12]:
lens=[]
for interact in InteractionIndexDict.keys():
    try:
        lens.append(len(VclusExp.Interact2VecInteract[interact]))
    except:
        repClust = InteractionRepClusDict[interact]
        assert VclusExp.clus2LenVecClus[VclusExp.clust2SpecClus[repClust][0]] == 0

In [13]:
len(lens)

144384

In [14]:
len(VclusExp.InteractVecInteracts), len(VclusExp.InteractVecVecs)

(5676, 5676)

In [15]:
numInteracts = len(InteractionIndexDict)
VecsInteracts = np.zeros((numInteracts, 3, 3))
VecGroupInteracts = np.full((numInteracts, 3), -1, dtype=int)

In [16]:
for interaction, repClus in InteractionRepClusDict.items():
    idx = InteractionIndexDict[interaction]
    # get the vector basis data here
    # if vector basis is empty, keep no of elements to -1.
    if VclusExp.clus2LenVecClus[VclusExp.clust2SpecClus[repClus][0]] == 0:
        continue
    vecList = VclusExp.clust2vecClus[repClus]
    vecList2 = VclusExp.Interact2VecInteract[interaction]
    
    vecsRepclust = [VclusExp.vecVec[i][j] for (i, j) in vecList]
    vecsInt = [VclusExp.InteractVecVecs[i][j] for (i, j) in vecList2]
    
    assert np.allclose(np.array(vecsRepclust), np.array(vecsInt))
    
    assert len(vecList) == MCJit.numVecsInteracts[idx]
    
    # store the vector
    for vecidx, tup in enumerate(vecList2):
        VecsInteracts[idx, vecidx, :] = VclusExp.InteractVecVecs[tup[0]][tup[1]].copy()
        VecGroupInteracts[idx, vecidx] = tup[0]

In [17]:
%%timeit
NVClusNoTrans = len(VclusExp.InteractVecVecs)
WBar, BBar = MCJit.ExpandLatGasNoTrans(state, jList, 
                                       dxList, offsc, 
                                       specRates, NVClusNoTrans,
                                       VecGroupInteracts,
                                       VecsInteracts,
                                       vacSiteInd)

6 s ± 36.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
